#### these scripts will mismatch original BERT vocab file

In [1]:
from vocab_mismatch_utils import *
from data_formatter_utils import *
from datasets import DatasetDict
from datasets import Dataset
import transformers
import pandas as pd

In [2]:
original_vocab = load_bert_vocab("../data-files/bert_vocab.txt")
corrupted_vocab = load_bert_vocab("../data-files/bert_vocab_mismatch.txt")

original_tokens = []
corrupted_tokens = []
for k, v in original_vocab.items():
    original_tokens.append(k)
for k, v in corrupted_vocab.items():
    corrupted_tokens.append(k)

token_mapping = dict(zip(original_tokens, corrupted_tokens))

In [3]:
original_tokenizer = transformers.BertTokenizer(
    vocab_file="../data-files/bert_vocab.txt")

def random_corrupt(example):
    original_sentence = example['text']
    original_tokens = original_tokenizer.tokenize(original_sentence)
    corrupted_tokens = []
    for ori_t in original_tokens:
        if ori_t in token_mapping.keys():
            cor_t = token_mapping[ori_t]
        else:
            cor_t = ori_t
        corrupted_tokens.append(cor_t)
    example['text'] = original_tokenizer.convert_tokens_to_string(corrupted_tokens)
    return example

#### Corrupted WikiText-103

In [ ]:
datasets = DatasetDict.load_from_disk("../data-files/wikitext-15M")
train_dataset = datasets["train"]
validation_dataset = datasets["validation"]
test_dataset = datasets["test"]

the following lines will corrupt the actual dataset, which takes long

In [ ]:
corrupted_train_dataset = train_dataset.map(random_corrupt)
corrupted_validation_dataset = validation_dataset.map(random_corrupt)
corrupted_test_dataset = test_dataset.map(random_corrupt)

In [ ]:
# save the corrupted datasets for pretraining!
datasets["train"] = corrupted_train_dataset
datasets["validation"] = corrupted_validation_dataset
datasets["test"] = corrupted_test_dataset
datasets.save_to_disk("../data-files/wikitext-15M-corrupted")

#### Corrupted SST3

In [4]:
sst_train_ternary = process_sst(os.path.join(sst_dirname, "train.txt"),
                                full_ternary_class_func)
sst_dev_ternary = process_sst(os.path.join(sst_dirname, "dev.txt"),
                              full_ternary_class_func,
                              include_subtrees=False)
sst_test_ternary = process_sst(os.path.join(sst_dirname, "test.txt"),
                               full_ternary_class_func,
                               include_subtrees=False)

In [26]:
# these are the original processed sst-tenary data
write_tsv(sst_train_ternary, output_filename=os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-train.tsv"))
write_tsv(sst_dev_ternary, output_filename=os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-dev.tsv"))
write_tsv(sst_test_ternary, output_filename=os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-test.tsv"))

In [6]:
# let us corrupt SST3 in the same way as before
train_df = pd.read_csv(os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-train.tsv"), delimiter="\t")
eval_df = pd.read_csv(os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-dev.tsv"), delimiter="\t")
test_df = pd.read_csv(os.path.join(external_output_dirname, "sst-tenary", "sst-tenary-test.tsv"), delimiter="\t")

train_df = Dataset.from_pandas(train_df)
eval_df = Dataset.from_pandas(eval_df)
test_df = Dataset.from_pandas(test_df)

corrupted_train_dataset = train_df.map(random_corrupt)
corrupted_validation_dataset = eval_df.map(random_corrupt)
corrupted_test_dataset = test_df.map(random_corrupt)

corrupted_datasets = DatasetDict({"train":corrupted_train_dataset, 
                                  "validation":corrupted_validation_dataset, 
                                  "test":corrupted_test_dataset})
corrupted_datasets.save_to_disk("../data-files/sst-tenary-corrupted")

#### Corrupted DynaSentv1.1 Round 1

#### Corrupted DynaSentv1.1 Round 2